In [2]:
import pulp

# Cost function

numLFA : ???
numLF: ???
numLE: ???


In [3]:
### Cost Function
 
def costFunction_bus(numLFA, numLF, numLE):
    return (numLFA * busData['LFA']['cost']) + (numLF * busData['LF']['cost']) + (numLE * busData['LE']['cost'])

battSize = 400 # in kWh
    
def costFunction_batt(numMP,numHP):
    return numMP*battData['MP']['price']*battSize + numHP*battData['HP']['price']*battSize

def costFunction_port():
    return 0

def costFunction_wages():
    return 0

def costFunction_charging():
    return 0

#combinationVariable['LFA']['A']['MP']['num']

def totalCostFunction(combinationVariable):
    
    # 1. Getting number of buses by type (LFA,LF,LE)
    numDict= {}
    numDict['LFA']=0
    numDict['LF']=0
    numDict['LE']=0
    for busKey in combinationVariable:
        for chargerType in combinationVariable[busKey]:
            for batType in combinationVariable[busKey][chargerType]:
                numDict[busKey] += combinationVariable[busKey][chargerType][batType]['num']
    
    # 2. Getting number of batteries in all buses by type (MP,HP)
    battDict= {}
    battDict['MP']=0
    battDict['HP']=0
    for busKey in combinationVariable:
        for chargerType in combinationVariable[busKey]:
            for battType in combinationVariable[busKey][chargerType]:
                battDict[battType] += combinationVariable[busKey][chargerType][battType]['num']
    
    # 1. Cost of buying Buses
    costBus = costFunction_bus(numDict['LFA'],numDict['LF'],numDict['LE'])
    
    # 2. Cost of buying Batteries in Buses
    costBatt = costFunction_batt(battDict['MP'],battDict['HP'])
    
    return costBus+costBatt

In [4]:
### Initializing Data (routes and batteries)

# Bus Data
busData = {
    'LFA': {
        'cost' : 570000,
        'capacity' : 140
    },
    'LF':{
        'cost' : 390000,
        'capacity' : 92
    },
    'LE': {
        'cost' : 350000,
        'capacity' : 60
    }
}

# Route Data
data = {
    'timeslot_1': {
        'time': 240,
        
        'route_1' : {
            'passengers' : 700,
            'bus_frequency' : 10,
            'distance' : 10,
            'duration' : 30
        },
        
        'route_2' : {
            'passengers' : 2000,
            'bus_frequency' : 10,
            'distance' : 10,
            'duration' : 40
        },
        'route_7': {
            'passengers' : 3300,
            'bus_frequency' : 10,
            'distance' : 10,
            'duration' : 40
        }

    },
    'timeslot_2': {
        'time': 360,
        
        'route_1' : {
            'passengers' : 700,
            'bus_frequency' : 10,
            'distance' : 10,
            'duration' : 30
        },
        
        'route_2' : {
            'passengers' : 300,
            'bus_frequency' : 20,
            'distance' : 10,
            'duration' : 40
        },
        'route_7': {
            'passengers' : 1000,
            'bus_frequency' : 10,
            'distance' : 10,
            'duration' : 40
        }
    }
}

# Battery Data

battData = {
    'HP': {
        'energyDensity': 60,
        'operatingV': 600,
        'maxChargeCur': 625,
        'maxDischargeCur': 500,
        'cycleLife': 6500,
        'price': 1150 # Euro per kWh
    },
    'MP':{
        'energyDensity': 7,
        'operatingV': 600,
        'maxChargeCur': 450,
        'maxDischargeCur': 450,
        'cycleLife': 3000,
        'price': 720 # Euro per kWh
    }
}

chargeData = {
    'A': {
        'chargeV': 500,
        'chargeCur': 60
    },
    'B': {
        'chargeV': 555,
        'chargeCur': 450
    },
    'C': {
        'chargeV': 500,
        'chargeCur': 900
    }
}


In [5]:
### Importing Modules

import pandas
import pulp


In [6]:
### Calculating max appearance from timeslot, route, battType, chargeType

def chargeDepletion(route,battType):
    dur = data['timeslot_1'][route]['duration']/60 # in hours
    
    #Assume maximum discharge current and max operating V
    dischargeRate = battData[battType]['maxDischargeCur']*battData[battType]['operatingV']
    return dur*dischargeRate
    
def rechargeDuration(chargeDepleted,battType,chargeType):
    rechargeRate1 = chargeData[chargeType]['chargeV']*chargeData[chargeType]['chargeCur'] # Charge rate pushed by charging port
    rechargeRate2 = chargeData[chargeType]['chargeV']*battData[battType]['maxChargeCur'] # Charge rate accepted by battery
    
    actualRechargeRate = max(rechargeRate1,rechargeRate2) 
    rechargeDur = chargeDepleted/actualRechargeRate
    return rechargeDur
                                            
def maxAppearanceLFA(route,timeslot,numLFA,battType,chargeType):
    timeslotDuration = data[timeslot]['time']
    roundtripDuration = data[timeslot][route]['duration']  
    
    # Calculating recharging time
    chargeDepleted = chargeDepletion(route,battType)
    rechargeDur = rechargeDuration(chargeDepleted,battType,chargeType)
    
    # Maximum number of appearance = number of bus * (rountrip duration + charging duration after roundtrip)
    maxAppearLFA = numLFA*timeslotDuration/(roundtripDuration+rechargeDur)
                                            
    return maxAppearLFA

def maxAppearanceLF(route,timeslot,numLF,battType,chargeType):
    timeslotDuration = data[timeslot]['time']
    roundtripDuration = data[timeslot][route]['duration']
    
    # Calculating recharging time
    chargeDepleted = chargeDepletion(route,battType)
    rechargeDur = rechargeDuration(chargeDepleted,battType,chargeType)
    
    maxAppearLF = numLF*timeslotDuration/(roundtripDuration+rechargeDur)
    return maxAppearLF

def maxAppearanceLE(route,timeslot,numLE,battType,chargeType):
    timeslotDuration = data[timeslot]['time']
    roundtripDuration = data[timeslot][route]['duration']
    
    # Calculating recharging time
    chargeDepleted = chargeDepletion(route,battType)
    rechargeDur = rechargeDuration(chargeDepleted,battType,chargeType)
    
    maxAppearLE = numLE*timeslotDuration/(roundtripDuration+rechargeDur)
    return maxAppearLE

# Main Algorithm

optimization of the number of buses (cost of the buses)

constraint1 : The cheapest bus should appear as much as possible
constraint2 : The buses should have enough capacity to carry the predicted number of people

In [47]:
def fill():
    chargingType = dict()
    for key in chargeData:
        if key not in chargingType:
            chargingType[key] = fillBat()
    return chargingType

i = 0
def fillBat():
    batEntry = dict()
    
    global counter
    
    """
    for key in battData:
        if key not in batEntry:
            batEntry[key] = dict()
            batEntry[key]['num'] = pulp.LpVariable("num", lowBound=0,cat="Integer")
            batEntry[key]['app'] = pulp.LpVariable("app" + str(i), lowBound=0,cat="Integer")
            i += 1      
    return batEntry
    """
    
    for key in battData:
        
        batEntry[key] = dict()
        batEntry[key]['num'] = pulp.LpVariable("num" + str(i), lowBound=0,cat="Integer")
        batEntry[key]['app'] = pulp.LpVariable("app" + str(i), lowBound=0,cat="Integer")
        i += 1      
        
    return batEntry


# Data structure :

{
    LFA:{
        A:{
            MA:{
                num:
                app
            }
            
            HP:{
                num:
                app
            }
        }
        
        B:{
        }
        ...
    }
}

In [48]:
def wrapTotalNum(route,timeslot,model,combinationVariable):
    totalNum = 0
    for key in combinationVariable:
        if key == 'LFA':
            fct_to_call = maxAppearanceLFA
        elif key == 'LF':
            fct_to_call = maxAppearanceLF
        else:
            fct_to_call = maxAppearanceLE
            
        for chargerType in combinationVariable[key]:
            for batType in combinationVariable[key][chargerType]:
                
                # Constraint 1 (Number of bus appearances)
                num = combinationVariable[key][chargerType][batType]['num']
                app = combinationVariable[key][chargerType][batType]['app']
                model += app <= fct_to_call(route, timeslot, num, batType, chargerType)
            
                #Constraint 2 (passenger)    
                totalNum = totalNum + num
                #totalCapacity = totalCapacity + (app * busData[key]['capacity'])
        return totalNum

def wrapCapacity(route,timeslot,model,combinationVariable):
    totalCapacity = 0
    for key in combinationVariable:
        if key == 'LFA':
            fct_to_call = maxAppearanceLFA
        elif key == 'LF':
            fct_to_call = maxAppearanceLF
        else:
            fct_to_call = maxAppearanceLE
            
        for chargerType in combinationVariable[key]:
            for batType in combinationVariable[key][chargerType]:
                
                # Constraint 1 (Number of bus appearances)
                num = combinationVariable[key][chargerType][batType]['num']
                app = combinationVariable[key][chargerType][batType]['app']
                model += app <= fct_to_call(route, timeslot, num, batType, chargerType)
            
                #Constraint 2 (passenger)    
                #totalNum = totalNum + num
                totalCapacity = totalCapacity + (app * busData[key]['capacity'])
        return totalCapacity

In [49]:
### Linear Programming Optimization

def solveLP(route,timeslot):

    # Instantiate Problem, Declaring holder variables
    model = pulp.LpProblem("Cost Optimisation", pulp.LpMinimize)
    totalNum = 0
    totalCapacity = 0
    
    #Creating JSON structure for all the buses
    combinationVariable = dict()
    
    
    for bus in busData:
        if bus not in combinationVariable:
            combinationVariable[bus] = fill()
            
    print(combinationVariable[bus])
    
    for key in combinationVariable:
        if key == 'LFA':
            fct_to_call = maxAppearanceLFA
        elif key == 'LF':
            fct_to_call = maxAppearanceLF
        else:
            fct_to_call = maxAppearanceLE
            
        for chargerType in combinationVariable[key]:
            for batType in combinationVariable[key][chargerType]:
                
                # Constraint 1 (Number of bus appearances)
                num = combinationVariable[key][chargerType][batType]['num']
                app = combinationVariable[key][chargerType][batType]['app']
                model += app <= fct_to_call(route, timeslot, num, batType, chargerType)
            
                #Constraint 2 (passenger)    
                totalNum = totalNum + num
                totalCapacity = totalCapacity + (app * busData[key]['capacity'])

    # Objective function
    model += totalCostFunction(combinationVariable), "Cost"

    # Constraint 1 (Number of bus appearances)
    model += wrapTotalNum(route,timeslot,model,combinationVariable) >= data[timeslot]['time']/data[timeslot][route]['bus_frequency']
    
    # Constraint 2 (Enough Passenger Capacity)

    model += wrapCapacity(route,timeslot,model,combinationVariable) >= data[timeslot][route]['passengers']
    
    # Solving
    
    #print(model)
    
    
    model.solve()
    
    printResult()
    
    print('For ' + timeslot + ", " + route + ":")
    
    print('Status: ' + str(pulp.LpStatus[model.status]))
    
    print('numLFA: {}'.format(combinationVariable['LFA']['A']['MP']['num'].varValue))
    
    print('Cost: ' + str(pulp.value(model.objective)))
    print('\n')
    
    
solveLP('route_1', 'timeslot_1')

{'A': {'HP': {'num': num12, 'app': app12}, 'MP': {'num': num13, 'app': app13}}, 'B': {'HP': {'num': num14, 'app': app14}, 'MP': {'num': num15, 'app': app15}}, 'C': {'HP': {'num': num16, 'app': app16}, 'MP': {'num': num17, 'app': app17}}}


NameError: name 'printResult' is not defined

# Initializing Problems

Initialize Across Different Routes and Timeslots

In [50]:
listRoutes = ['route_1','route_2','route_3','route_4','route_5','route_6','route_7','route_8','route_9','route_10','route_11',
             'route_12','route_13']
listTimeslots = ['timeslot_1','timeslot_2','timeslot_3','timeslot_4']


solveLP('route_1', 'timeslot_1')
solveLP('route_2', 'timeslot_1')
solveLP('route_7', 'timeslot_1')

solveLP('route_1', 'timeslot_2')
solveLP('route_2', 'timeslot_2')
solveLP('route_7', 'timeslot_2')

{'A': {'HP': {'num': num30, 'app': app30}, 'MP': {'num': num31, 'app': app31}}, 'B': {'HP': {'num': num32, 'app': app32}, 'MP': {'num': num33, 'app': app33}}, 'C': {'HP': {'num': num34, 'app': app34}, 'MP': {'num': num35, 'app': app35}}}


NameError: name 'printResult' is not defined